In [8]:
import requests

search_keyword = '청바지'
url = f'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={search_keyword}'
#url = f'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search_keyword}&start=11'

result = requests.get(url)
#print(result)

from bs4 import BeautifulSoup
soup = BeautifulSoup(result.text, 'html.parser')
#print(soup)
print("================")

news_titles = soup.select('.sp_nnews .news_wrap .news_tit')
print(news_titles)

print('총 뉴스 제목 개수: ', len(news_titles))
for item in news_titles:
    print(item['title'])

[<a class="news_tit" href="http://www.sportsseoul.com/news/read/1148706?ref=naver" onclick="return goOtherCR(this, 'a=nws*a.tit&amp;r=1&amp;i=88156f78_000000000000000000871611&amp;g=468.0000871611&amp;u='+urlencode(this.href));" target="_blank" title="용인시 진·캐주얼데이 '눈길'... 이상일 청바지 출근">용인시 진·캐주얼데이 '눈길'... 이상일 <mark>청바지</mark> 출근</a>, <a class="news_tit" href="https://www.yna.co.kr/view/AKR20220805056900061?input=1195m" onclick="return goOtherCR(this, 'a=nws*e.tit&amp;r=5&amp;i=880000D8_000000000000000013356266&amp;g=001.0013356266&amp;u='+urlencode(this.href));" target="_blank" title="&quot;복장 자유롭고 편하게&quot;…용인시, 매주 금요일 '진·캐주얼데이'">"복장 자유롭고 편하게"…용인시, 매주 금요일 '진·캐주얼데이'</a>, <a class="news_tit" href="https://www.sedaily.com/NewsView/269OILNHQZ" onclick="return goOtherCR(this, 'a=nws*e.tit&amp;r=6&amp;i=88000108_000000000000000004084338&amp;g=011.0004084338&amp;u='+urlencode(this.href));" target="_blank" title="'매주 금요일 가벼운 옷차림 출근'…이상일 용인시장,‘진·캐주얼데이’운영">'매주 금요일 가벼운 옷차림 출근'…이상일 용인시장,‘진·캐주얼데이’운영<

In [ ]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import re
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''
< naver 뉴스 검색시 리스트 크롤링하는 프로그램 > _select사용
- 크롤링 해오는 것 : 링크,제목,신문사,날짜,내용요약본
- 날짜,내용요약본  -> 정제 작업 필요
- 리스트 -> 딕셔너리 -> df -> 엑셀로 저장 
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''

#각 크롤링 결과 저장하기 위한 리스트 선언 
title_text=[]
link_text=[]
source_text=[]
date_text=[]
contents_text=[]
result={}

#엑셀로 저장하기 위한 변수
RESULT_PATH ='C:/Users/yyk834/study/dataset/'  #결과 저장할 경로
now = datetime.now() #파일이름 현 시간으로 저장하기

#날짜 정제화 함수
def date_cleansing(test):
    try:
        #지난 뉴스
        #머니투데이  10면1단  2018.11.05.  네이버뉴스   보내기  
        pattern = '\d+.(\d+).(\d+).'  #정규표현식 
    
        r = re.compile(pattern)
        match = r.search(test).group(0)  # 2018.11.05.
        date_text.append(match)
        
    except AttributeError:
        #최근 뉴스
        #이데일리  1시간 전  네이버뉴스   보내기  
        pattern = '\w* (\d\w*)'     #정규표현식 
        
        r = re.compile(pattern)
        match = r.search(test).group(1)
        #print(match)
        date_text.append(match)


#내용 정제화 함수 
def contents_cleansing(contents):
    first_cleansing_contents = re.sub('<dl>.*?</a> </div> </dd> <dd>', '', 
                                      str(contents)).strip()  #앞에 필요없는 부분 제거
    second_cleansing_contents = re.sub('<ul class="relation_lst">.*?</dd>', '', 
                                       first_cleansing_contents).strip()#뒤에 필요없는 부분 제거 (새끼 기사)
    third_cleansing_contents = re.sub('<.+?>', '', second_cleansing_contents).strip()
    contents_text.append(third_cleansing_contents)
    #print(contents_text)
    

def crawler(maxpage,query,sort,s_date,e_date):

    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1  
    maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지
    
    while page <= maxpage_t:
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        
        response = requests.get(url)
        html = response.text
 
        #뷰티풀소프의 인자값 지정
        soup = BeautifulSoup(html, 'html.parser')
 
        #<a>태그에서 제목과 링크주소 추출
        atags = soup.select('.news_tit')
        for atag in atags:
            title_text.append(atag.text)     #제목
            link_text.append(atag['href'])   #링크주소
            
        #신문사 추출
        source_lists = soup.select('.info_group > .press')
        for source_list in source_lists:
            source_text.append(source_list.text)    #신문사
        
        #날짜 추출 
        date_lists = soup.select('.info_group > span.info')
        for date_list in date_lists:
            # 1면 3단 같은 위치 제거
            if date_list.text.find("면") == -1:
                date_text.append(date_list.text)
        
        #본문요약본
        contents_lists = soup.select('.news_dsc')
        for contents_list in contents_lists:
            contents_cleansing(contents_list) #본문요약 정제화
        

        #모든 리스트 딕셔너리형태로 저장
        result= {"date" : date_text , "title":title_text ,  "source" : source_text ,"contents": contents_text ,"link":link_text }  
        print(page)
        
        df = pd.DataFrame(result)  #df로 변환
        page += 10
    
    
    # 새로 만들 파일이름 지정
    outputFileName = '%s-%s-%s  %s시 %s분 %s초 merging.xlsx' % (now.year, now.month, now.day, now.hour, now.minute, now.second)
    df.to_excel(RESULT_PATH+outputFileName,sheet_name='sheet1')
    
    

def main():
    info_main = input("="*50+"\n"+"입력 형식에 맞게 입력해주세요."+"\n"+" 시작하시려면 Enter를 눌러주세요."+"\n"+"="*50)
    
    maxpage = input("최대 크롤링할 페이지 수 입력하시오: ")  
    query = input("검색어 입력: ")  
    sort = input("뉴스 검색 방식 입력(관련도순=0  최신순=1  오래된순=2): ")    #관련도순=0  최신순=1  오래된순=2
    s_date = input("시작날짜 입력(2019.01.04):")  #2019.01.04
    e_date = input("끝날짜 입력(2019.01.05):")   #2019.01.05
    
    crawler(maxpage,query,sort,s_date,e_date) 
    
main()

In [2]:
import pandas as pd
import os

dataframe =  pd.read_excel('../study/dataset/2022-8-4  23시 33분 16초 merging.xlsx', header=None, encoding='CP949', nrows=2500)
news_contents = dataframe.iloc[:, 2].tolist()

if os.path.isfile('../study/dataset/study_crawling_SectionDataset.xlsx'):
    df = pd.read_excel('../study/dataset/study_crawling_SectionDataset.xlsx' , names=['contents','section'], header=0)

else:
    bodyStruc = {'contents': [], 'section': []}
    df = pd.DataFrame( bodyStruc, columns=['contents', 'section'] )


print(df.columns)
print(news_contents)

df.head()

for item in news_contents:
    texts = item
    texts = texts.replace(',', "")
    texts = texts.replace('\'', "")
    texts = texts.replace('\"', "")
    texts = texts.replace('…', " ")
    print(texts)
    oneData = {'contents': texts, 'section': '0'}
    df = df.append( oneData, ignore_index=True )

df.to_excel('../study/dataset/study_crawling_SectionDataset.xlsx', encoding='UTF-8', index=False, header=True)

Index(['contents', 'section'], dtype='object')
['title', "한예슬, 청바지 남을 만큼 납작 개미허리…파격 패션도 '찰떡'", '강민경, 세젤예 청바지핏..사복여신의 청순 자태', '한예슬, 파격 섹시 클리비지룩..잘록 개미허리에 청바지 헐렁', "베리베리 호영, 독특한 청바지 '아이돌 룩~'", '벨라, 카이아, 켄달처럼 청바지 입기', '남윤수 ‘안경+셔츠+청바지+양말+구두’ 모든게 다 완벽[포토엔HD]', "하니, 트위드 재킷와 청바지 '상큼 여친룩'", "김세정 '통 큰 청바지 패션' [MK포토]", '“사복 여신 클라쓰” 청바지 브랜드와 콜라보한 차정원, 대박 쳤다', '님아 청바지 살펴보소, 50만 위구르인의 피 묻은 솜 들어 있는지', '정장 벗고 청바지 입은 구현모 "KT, \'콘텐츠\'로 디지코 변화 일군다"', '\'청바지 노타이\' KT 구현모 "콘텐츠는 필수…이제는 돈 벌 때"', '청바지 입은 구현모 "K-콘텐츠로 글로벌 시장 판도 변화"', "빨간 넥타이 맨 안철수, 정장에 반기 든 美 초선의원...'복장 정치'는 무엇", "방탄소년단 지민, 천사인줄 알았더니 '숲의 요정'..청바지에 흰 티로 '여심 올...", '\'발리댁\' 가희, 청바지 입고 늘씬 각선미.."서핑은 비키니 입어도 골프는 안타...', '“넉넉하게 여유롭게” 스키니진 가고 와이드데님 온다', "초아, 허리 살짝 드러내고…티셔츠+청바지 룩도 '완벽'", '‘청바지’ 구현모 “KT, 이제 콘텐츠로 돈 벌 때…국내 최대 ‘4000억’ 투자...', '인도 여성들 ‘찢어진 청바지 인증샷’ 챌린지 열풍...왜?', '김세정, 흰 티에 청바지만 입어도 눈부셔..청순美 뚝뚝', "김사랑, 티셔츠+청바지 '꾸안꾸' 패션…일상이 화보네", '“4.5%↑”…‘청바지’ 차림 구현모 진정성에 KT 주주들이 움직였다', "KT, '넷플릭스급' 투자···1,300만 쓰는 플랫폼에 대작 띄운다", '청바지에 갈색구두 구현모 "드레스 코드요? 맞습니다

In [5]:
import pandas as pd
import os

dataframe =  pd.read_excel('../study/dataset/2022-8-4  23시 33분 16초 merging.xlsx', header=None, encoding='CP949', nrows=2500)
news_contents = dataframe.iloc[:, 2].tolist()

if os.path.isfile('../study/dataset/study_crawling_SectionDataset.xlsx'):
    df = pd.read_excel('../study/dataset/study_crawling_SectionDataset.xlsx' , names=['contents','section'], header=0)

else:
    bodyStruc = {'contents': [], 'section': []}
    df = pd.DataFrame( bodyStruc, columns=['contents', 'section'] )


# print(df.columns)
# print(news_contents)

df.head()

,contents,section
0,title,0
1,한예슬 청바지 남을 만큼 납작 개미허리 파격 패션도 찰떡,0
2,강민경 세젤예 청바지핏..사복여신의 청순 자태,0
3,한예슬 파격 섹시 클리비지룩..잘록 개미허리에 청바지 헐렁,0
4,베리베리 호영 독특한 청바지 아이돌 룩~,0


In [14]:
from konlpy.tag import Okt
okt = Okt()
result =[]
result = [okt.nouns(i) for i in news_contents] #명사만 추출
final_result= [r for i in result for r in i]
print(final_result)

['한예슬', '청바지', '남', '만큼', '납작', '개미', '허리', '파격', '패션', '찰떡', '강민경', '젤예', '청바지', '핏', '사복', '신의', '청순', '자태', '한예슬', '파격', '섹시', '클리', '비지', '개미', '허리', '청바지', '헐렁', '베리', '베리', '호영', '청바지', '아이돌', '벨라', '카이', '켄달', '청바지', '남윤수', '안경', '셔츠', '청바지', '양말', '구두', '모든', '완벽', '포토', '하니', '트', '위드', '재킷', '청바지', '상큼', '여친', '김세정', '통', '청바지', '패션', '포토', '사복', '여신', '청바지', '브랜드', '콜라보', '차', '정원', '대박', '님', '청바지', '위구르인', '피', '솜', '정장', '청바지', '현모', '콘텐츠', '로', '디', '지코', '변화', '청바지', '노', '타이', '현모', '콘텐츠', '필수', '이제', '돈', '벌', '때', '청바지', '현모', '콘텐츠', '글로벌', '시장', '판도', '변화', '넥타이', '맨', '안철수', '정장', '반기', '초선', '의원', '복장', '정치', '무엇', '방탄소년단', '지민', '천사', '줄', '숲', '요정', '청바지', '티', '여심', '발리', '댁', '가희', '청바지', '각선미', '서핑', '비키니', '골프', '안타', '스키니진', '와이드', '데님', '초아', '허리', '살짝', '티셔츠', '청바지', '룩', '완벽', '청바지', '현모', '이제', '콘텐츠', '돈', '벌', '때', '국내', '최대', '투자', '인도', '여성', '청바지', '인증샷', '린지', '열풍', '왜', '김세정', '티', '청바지', '청순', '뚝뚝', '김사랑', '티셔츠', '청바지', '패션', '일상', '화보', '청바지', '

In [15]:
###텍스트에서 많이 나온 단어###
import pandas as pd
##한국어##
korean = pd.Series(final_result).value_counts().head(10)
print("한국어 top 10")
print(korean)

한국어 top 10
청바지    94
포토     18
핏      16
티      12
패션     10
얼       6
비주      6
현모      6
화보      6
허리      6
dtype: int64
